In [1]:
import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display
from torch import nn

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

In [2]:
language = 'ru'
speaker = 'kseniya_16khz'
device = torch.device('cuda:0')
model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                      model='silero_tts',
                                                                      language=language,
                                                                      speaker=speaker)
model = model.to(device)

Using cache found in /home/jovyan/.cache/torch/hub/snakers4_silero-models_master


In [3]:
torch.jit.script(model.tacotron).code

'def forward(self,\n    inputs: Tensor,\n    max_decoder_steps: Optional[int]=None) -> Tuple[Tensor, Tensor]:\n  embedding = self.embedding\n  embedded_inputs = torch.transpose((embedding).forward(inputs, ), 1, 2)\n  encoder = self.encoder\n  encoder_outputs = (encoder).forward(embedded_inputs, )\n  decoder = self.decoder\n  _0 = (decoder).forward(encoder_outputs, max_decoder_steps, )\n  mel_outputs, gate_outputs, alignments, mel_lengths, = _0\n  postnet = self.postnet\n  mel_outputs_postnet = (postnet).forward(mel_outputs, )\n  mel_outputs_postnet0 = torch.add(mel_outputs, mel_outputs_postnet)\n  if torch.gt(torch.size(inputs, 0), 1):\n    mel_outputs_postnet2 = (self).parse_output(mel_outputs_postnet0, mel_lengths, )\n    mel_outputs_postnet1 = mel_outputs_postnet2\n  else:\n    mel_outputs_postnet1 = mel_outputs_postnet0\n  return (mel_outputs_postnet1, mel_lengths)\n'

In [4]:
from typing import Optional, Tuple
from torch import Tensor


def forward(self,
            inputs: Tensor,
            max_decoder_steps: Optional[int] = None) -> Tuple[Tensor, Tensor]:
    embedding = self.embedding
    embedded_inputs = torch.transpose((embedding).forward(inputs, ), 1, 2)
    encoder = self.encoder
    encoder_outputs = (encoder).forward(embedded_inputs, )
    decoder = self.decoder
    _0 = (decoder).forward(encoder_outputs, max_decoder_steps, )
    
    mel_outputs, gate_outputs, alignments, mel_lengths, = _0
    postnet = self.postnet
    mel_outputs_postnet = (postnet).forward(mel_outputs, )
    mel_outputs_postnet0 = torch.add(mel_outputs, mel_outputs_postnet)
    
    if torch.gt(torch.size(inputs, 0), 1):
        mel_outputs_postnet2 = (self).parse_output(mel_outputs_postnet0, mel_lengths, )
        mel_outputs_postnet1 = mel_outputs_postnet2
    else:
        mel_outputs_postnet1 = mel_outputs_postnet0
    return (mel_outputs_postnet1, mel_lengths)  #%%

In [5]:
example_text = 'В недрах тундры выдры в гетрах тырят в вёдра ядра кедров. '
cleaned_example_text = "".join([c if c in symbols else " " for c in example_text])

speaker_id = torch.tensor(0).to(device)
embeder = model.tacotron.embedding.to(device)
encoder = model.tacotron.encoder.to(device)
decoder = model.tacotron.decoder.to(device)

input_tensor = torch.LongTensor([symbols.index(c) for c in cleaned_example_text]).unsqueeze(0).to(device)
embedded_inputs = embeder(input_tensor)
print(embedded_inputs.shape)

embedded_inputs = torch.transpose(embedded_inputs, 1, 2)
encoder_output = encoder(embedded_inputs)
print(encoder_output.shape)

mel_outputs, gate_outputs, alignments, mel_lengths = decoder(encoder_output)

torch.Size([1, 58, 512])


/home/jovyan/venvs/sinara/lib/python3.10/site-packages/torch/nn/modules/module.py:1527: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  return forward_call(*args, **kwargs)


torch.Size([1, 58, 512])


In [6]:
print("mel_outputs", mel_outputs.shape)  # <- use this as speech data representation
print("gate_outputs", gate_outputs.shape)
print("alignments", alignments.shape)
print("mel_lengths", mel_lengths.shape)

mel_outputs torch.Size([1, 128, 1000])
gate_outputs torch.Size([1, 1000])
alignments torch.Size([1, 1000, 58])
mel_lengths torch.Size([1])


In [7]:
output = model.vocoder(mel_outputs)
output

tensor([[[-0.0003, -0.0002, -0.0003,  ...,  0.0095,  0.0096,  0.0094]]],
       device='cuda:0')

In [8]:
output.shape

torch.Size([1, 1, 200000])

In [9]:
import IPython.display as ipd  # type: ignore

def display_audio(samples: torch.Tensor, sample_rate: int):
    """Renders an audio player for the given audio samples.

    Args:
        samples (torch.Tensor): a Tensor of decoded audio samples
            with shapes [B, C, T] or [C, T]
        sample_rate (int): sample rate audio should be displayed with.
    """
    assert samples.dim() == 2 or samples.dim() == 3

    samples = samples.detach().cpu()
    if samples.dim() == 2:
        samples = samples[None, ...]

    for audio in samples:
        ipd.display(ipd.Audio(audio, rate=sample_rate))

In [10]:
display_audio(output.detach().cpu(), sample_rate=14000)

In [18]:
mel_outputs.reshape(1, 1000, 128).shape

torch.Size([1, 1000, 128])

In [19]:
from open_musiclm.config import load_model_config

model_config = "./configs/model/musiclm_large_small_context.json"
model_config = load_model_config(model_config)

import torch
from open_musiclm.config import create_encodec_from_config
                                 
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

FileNotFoundError: [Errno 2] No such file or directory: './configs/model/musiclm_large_small_context.json'

In [ ]:
encodec_wrapper = create_encodec_from_config(model_config, device)
print("encodec успешно загружен")

wave = encodec_wrapper.decode_from_codebook_indices(mel_outputs)
wave = rearrange(wave, 'b 1 n -> b n')
wave